In [94]:
%pip install -q \
    fuzzywuzzy \
    easyocr \
    pandas \
    requests

debug = False


[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [95]:
import requests

def writeToEndpoint(endpoint, json):
    url = f'http://localhost:3000/api/{endpoint}'
    response = requests.post(url, json=json)
    if response.status_code == 200:
        if debug:
            print(f'Succes writing {endpoint}. Response:\n{response.text}')
    else:
        # if debug:
        print(f'\nError writing {endpoint}. \nRequest:{json}\nResponse:\n{response.text}')
    return response


In [96]:
# seed reference items using api

# import from csv
import pandas as pd

referenceitems_df = pd.read_csv('referenceitems.csv')

for index, row in referenceitems_df.iterrows():
    response = writeToEndpoint('referenceItem', {
        "name": row['name'],
        "quantity": row['quantity'],
        "unitOfMeasure": row['unitofmeasure'],
        "price": row['price'],
        "pricePerWeight": row['priceperweight'],
        "referenceUrl": row['referenceurl']
    })
    new_referenceitem_id = response.json()['id']
    referenceitems_df.at[index, 'id'] = int(new_referenceitem_id)
referenceitems_df['id'] = referenceitems_df['id'].astype(int)
referenceitems_df.set_index('id', inplace=True)
# referenceitems_df = referenceitems_df[['id', 'name', 'quantity', 'unitofmeasure', 'price', 'priceperweight', 'referenceurl']]
referenceitems_df

# test
# writeReferenceItem({
#     "name": "Sample item",
#     "quantity": 10,
#     "unitOfMeasure": "g",
#     "price": 100,
#     "pricePerWeight": 10,
#     "referenceUrl": "test.com"
# })

# import from queryclassification.csv to panda dataframe
queryclassification_df = pd.read_csv('queryclassification.csv')
queryclassification_df.reset_index(drop=True, inplace=True)
queryclassification_df['id'] = queryclassification_df.index
queryclassification_df.set_index('id', inplace=True)
queryclassification_df

temp_referenceItem_map = referenceitems_df.copy()
temp_referenceItem_map['id'] = temp_referenceItem_map.index
temp_referenceItem_map

queryclassification_df['referenceItemId'] = queryclassification_df['referenceItem'].map(temp_referenceItem_map.set_index('name')['id'].to_dict())
try:
    queryclassification_df['referenceItemId'] = queryclassification_df['referenceItemId'].astype(int)
except:
    print('Failed to convert referenceItemId to int')
    print(queryclassification_df[queryclassification_df['referenceItemId'].isnull()])
queryclassification_df.drop(columns=['referenceItem'], inplace=True)
queryclassification_df

,query,referenceItemId
id,,
0,3x KIND Oats & Honey Granola with Toasted Coco...,14
1,Annie's Homegrown Gluten-Free Rice Pasta & Che...,19
2,Annie's Homegrown Gluten-Free Rice Pasta White...,19
3,Annies rice pasta and white cheddar,19
4,Baldstreet all beef frankfurters,40
...,...,...
169,Mac & Cheese,19
170,Caramel Chocolate Chip Cookie Bar,34
171,Purest White Bread,32


In [97]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import re
import easyocr
import pandas as pd
import shutil
import numpy as np
import os

reader = easyocr.Reader(['en']) # this needs to run only once to load the model into memory

def ocrImage(image_id, image_path):
    receipt_strings = reader.readtext(image_path)

    # guard clause
    if len(receipt_strings) < 1:
        return # no text found

    # transformation
    receipt_strings_df = pd.DataFrame(receipt_strings, columns=['boundingBox', 'text', 'confidence'])
    receipt_strings_df.drop(columns=['confidence'], inplace=True)
    receipt_strings_df['imageFileId'] = image_id
    # receipt_strings_df['imageFileId'] = receipt_strings_df['imageFileId'].astype(int)

    # push to API
    responses = []
    for index, row in receipt_strings_df.iterrows():
        # print(row['text'])
        # print(row['boundingBox'])
        # print(row['imageFileId'])
        boundingBox_str = [[int(value) for value in sublist] for sublist in row['boundingBox']]
        response = writeToEndpoint('receiptText', {
            'text': str(row['text']), 
            'boundingBox': str(boundingBox_str), 
            'imageFileId': int(row['imageFileId'])
        })
        responses.append(response.json())
    return responses

choices_dict = queryclassification_df[['query']].to_dict()['query']

def parseReceiptTextsForEligibleExpenses(receipt_texts, receipt_id):
    # receipt = fetch_receipt(receipt_id)
    # receipt_texts = receipt['data']['receipt']['receiptTexts']
    threshold = 80
    
    def clean_text(text):
        text = re.sub(r'^\d{1,10}', '', text)
        if len(text) <= 2:
            return ''
        if not any(char.isalnum() for char in text):
            return ''
        return text

    for receipt_text in receipt_texts:
        receipt_text_id = receipt_text['id']
        receipt_text = receipt_text['text']
        clean_receipt_text = clean_text(receipt_text)
        if clean_text(clean_receipt_text) == '':
            continue
        best_match, best_score, best_match_id = process.extractOne(clean_receipt_text, choices_dict, scorer=fuzz.token_set_ratio)

        if best_score >= threshold:
            # avoid strict substring matches
            if len(clean_receipt_text)/len(best_match) < 0.4:
                continue
            # found a decent match
            # get corresponding referenceItem 
            reference_item_id = int(queryclassification_df.loc[best_match_id]['referenceItemId'])
            # create eligible expense
            # print(f'Creating expense for {receipt_text} with reference item id {reference_item_id}')
            writeToEndpoint('expense',{
                    'receiptTextId': receipt_text_id,
                    'referenceItemId': reference_item_id,
                    'receiptId': receipt_id,
                    'priceEach': 0, # TODO: find price
                    'quantity': 1, # TODO: find quantity
                }
            )

def upload_image(image_path, receipt_id):
    # fake upload, i.e. copy to frontend folder
    destination_folder = "../receipts-app/public/uploads/"
    shutil.copy(image_path, destination_folder)
    image_url = destination_folder + image_path.split("/")[-1]
    response = writeToEndpoint('imageFile', {
        "url": image_url, 
        "receiptId": receipt_id,
        })
    return response
# test
# upload_image('./receipts/IMG_4553.jpg')

# TODO: batch multiple images in array
def main_single_receipt(image_path):
    receipt_id = writeToEndpoint('receipt', {}).json()['id']
    image_id = upload_image(image_path, receipt_id).json()['id']
    receiptTexts = ocrImage(image_id, image_path) # for notebooks only, using local file
    # print(receiptTexts)
    print(f'Parsed receipt {receipt_id}: {receiptTexts}', end='\r')
    if receiptTexts is None:
        return None
    parseReceiptTextsForEligibleExpenses(receiptTexts, receipt_id)
    # ... need to match with reference items
    # createReceipt(receiptTexts)
    # return receiptID

# main_single_receipt('./receipts/IMG_4553.jpg')


In [98]:
def list_files(directory, extension):
    return list(f for f in os.listdir(directory) if f.endswith('.' + extension))

all_files = list_files('./receipts', 'jpg')

for file in all_files:
    main_single_receipt(f'./receipts/{file}')

Parsed receipt 9: [{'id': 523, 'text': '3 APPLES', 'boundingBox': '[[380, 498], [653, 498], [653, 618], [380, 618]]', 'imageFileId': 9, 'createdAt': '2024-08-02T02:28:28.815Z', 'updatedAt': '2024-08-02T02:28:28.815Z'}, {'id': 524, 'text': '2544 Cointy Road No 29', 'boundingBox': '[[303, 650], [723, 650], [723, 699], [303, 699]]', 'imageFileId': 9, 'createdAt': '2024-08-02T02:28:28.820Z', 'updatedAt': '2024-08-02T02:28:28.820Z'}, {'id': 525, 'text': 'Pakenham; Ontario', 'boundingBox': '[[356, 698], [667, 698], [667, 741], [356, 741]]', 'imageFileId': 9, 'createdAt': '2024-08-02T02:28:28.824Z', 'updatedAt': '2024-08-02T02:28:28.824Z'}, {'id': 526, 'text': 'KOA 2X0', 'boundingBox': '[[437, 741], [592, 741], [592, 784], [437, 784]]', 'imageFileId': 9, 'createdAt': '2024-08-02T02:28:28.829Z', 'updatedAt': '2024-08-02T02:28:28.829Z'}, {'id': 527, 'text': 'Inconceivably Gluten-Freel', 'boundingBox': '[[291, 800], [732, 800], [732, 843], [291, 843]]', 'imageFileId': 9, 'createdAt': '2024-08-02

KeyboardInterrupt: 